<a href="https://colab.research.google.com/github/TunaInABottle/data_mining_2022/blob/main/Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/TunaInABottle/data_mining_2022.git

Cloning into 'data_mining_2022'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 108 (delta 43), reused 82 (delta 25), pack-reused 0
Receiving objects: 100% (108/108), 48.72 KiB | 8.12 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

In [21]:
!pip install turicreate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
from __future__ import print_function # to conform python 2.x print to python 3.x
import turicreate
import sys
import os
from scipy.sparse import csr_matrix
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

%matplotlib inline

In [23]:
df1= turicreate.SFrame.read_csv("/content/data_mining_2022/file_gen/queries_content.csv")

Finished parsing file /content/data_mining_2022/file_gen/queries_content.csv

Parsing completed. Parsed 100 lines in 0.075486 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 100 lines. Lines per second: 3433.71

Finished parsing file /content/data_mining_2022/file_gen/queries_content.csv

Parsing completed. Parsed 100 lines in 0.039613 secs.

In [24]:
df1

Surname,Name,City,Profession,Age
Smith,Abigail,Amsterdam,accountant,5
Johnson,Adam,Athens,actor,6
Sullivan,Wojchek,Auckland,web developer,99
Williams,Aiden,Baku,acupuncturist,7
Simmons,Scarlett,Bangkok,public relationsspecialist ...,86
Jones,Adriana,Barcelona,administrator,8
Griffin,Teddy,Beijing,software developer,92
Brown,Ainsley,Belgrade,adult education teacher,9
Davis,Alex,Berlin,advertising executive,10
Garcia,Alexander,Birmingham,aerospace engineer,11


TF-IDF

In [25]:
df1['tf_idf'] = turicreate.text_analytics.tf_idf(df1['Profession'])

In [26]:
df1['tf_idf']

dtype: dict
Rows: 100
[{'accountant': 4.605170185988092}, {'actor': 4.605170185988092}, {'developer': 3.912023005428146, 'web': 4.605170185988092}, {'acupuncturist': 4.605170185988092}, {'relations': 4.605170185988092, 'specialist': 3.912023005428146, 'public': 4.605170185988092}, {'administrator': 4.605170185988092}, {'developer': 3.912023005428146, 'software': 4.605170185988092}, {'education': 4.605170185988092, 'teacher': 3.912023005428146, 'adult': 4.605170185988092}, {'executive': 4.605170185988092, 'advertising': 4.605170185988092}, {'engineer': 2.302585092994046, 'aerospace': 4.605170185988092}, {'engineer': 2.302585092994046, 'agricultural': 3.506557897319982}, {'scientist': 2.995732273553991, 'agricultural': 3.506557897319982}, {'technologist': 3.2188758248682006, 'agricultural': 3.506557897319982}, {'controller': 4.605170185988092, 'traffic': 4.605170185988092, 'air': 4.605170185988092}, {'mechanic': 4.605170185988092, 'aircraft': 4.605170185988092}, {'technologist': 3.218875

In [27]:
def sframe_to_scipy(x, column_name):
    '''
    Convert a dictionary column of an SFrame into a sparse matrix format where
    each (row_id, column_id, value) triple corresponds to the value of
    x[row_id][column_id], where column_id is a key in the dictionary.
       
    Example
    >>> sparse_matrix, map_key_to_index = sframe_to_scipy(sframe, column_name)
    '''
    assert type(x[column_name][0]) == dict, \
        'The chosen column must be dict type, representing sparse data.'
    
    # 1. Add a row number (id)
    x = x.add_row_number()

    # 2. Stack will transform x to have a row for each unique (row, key) pair.
    x = x.stack(column_name, ['feature', 'value'])

    # Map feature words to integers 
    unique_words = sorted(x['feature'].unique())
    mapping = {word:i for i, word in enumerate(unique_words)}
    x['feature_id'] = x['feature'].apply(lambda x: mapping[x])

    # Create numpy arrays that contain the data for the sparse matrix.
    row_id = np.array(x['id'])
    col_id = np.array(x['feature_id'])
    data = np.array(x['value'])
    
    width = x['id'].max() + 1
    height = x['feature_id'].max() + 1
    
    # Create a sparse matrix.
    mat = csr_matrix((data, (row_id, col_id)), shape=(width, height))
    return mat, mapping

In [28]:
tf_idf, map_index_to_word = sframe_to_scipy(df1, 'tf_idf')

In [29]:
tf_idf.shape

(100, 115)

In [30]:
from sklearn.preprocessing import normalize
tf_idf = normalize(tf_idf)

<h4>Here we will work with the Euclidean distance (normalized, it will be equivalent to the cosine distance). <br>
In the data cleaning part, we want to transform the <strong> City</strong> and <strong> Profession </strong> columns into numerical data, to be useful for the similarity measure, for this we need two datasets: <br>
<ul>
<li>A dataset of culturally similar cities (because people from same cultures tend to have similar tastes, behaviors...) </li>
<li>A dataset of similar occupations (we can transform the Profession column into categorical data (science, art, law), then into numerical data) </li>
</ul>



<h4>For the similarity measurement, two options are available: <br>
<ul>
<li>We can have a symmetric square matrix S of size n²=length(df)², indication the similarity between each two queries. <br>
But, calculating the S matrix will be computationally costly, at best complexity of O(n²) </li>
<li>Clustering  </li>
</h4>

In [3]:
df= pd.read_csv("/content/data_mining_2022/file_gen/queries_content.csv")

In [4]:
df

,Surname,Name,City,Profession,Age
0,Smith,Abigail,Amsterdam,accountant,5
1,Johnson,Adam,Athens,actor,6
2,Sullivan,Wojchek,Auckland,web developer,99
3,Williams,Aiden,Baku,acupuncturist,7
4,Simmons,Scarlett,Bangkok,public relations specialist,86
...,...,...,...,...,...
95,Wallace,Kek,Wellington,writer,100
96,Brooks,Elizabeth,Wroclaw,market researcher,67
97,Kelly,Ella,Yerevan,marketing specialist,68
98,Sanders,Emily,Zagreb,massage therapist,69


<h4>For now, we will only work with the <strong>Age </strong> column, the rest of the work will remain the same since we should only change the formula for the Euclidean distance.
</h4>

<h2> S Matrix
</h2>

In [118]:
#the S matrix will contain distances of each query to the other queries
#then we normalize rows of the S matrix
#When people talk about generations, they're referring to all those born within roughly a 20-year period
#So we will set 20 as a threshold to similarity 

def S_Matrix (df, n): 
  #n: index of the column "Age"
  S=np.zeros(df.shape[0]**2).reshape(df.shape[0], df.shape[0])
  for i in range (df.shape[0]):
    S[i,i]=100
    for j in range (i+1,df.shape[0]):
      if (df.iloc[i][n] != 'Null') and (df.iloc[j][n] != 'Null') :
        S[i,j]=abs(df.iloc[i][n] - df.iloc[j][n])
        S[j,i]=S[i,j]
      else :
        S[i,j]=100
        S[j,i]=100
  return (S)
        
""" 
  a=S.sum(1)   #we will use these values, to normalize the value 20 (threshold) for each query, 
              #and we will put it in S[i,i]

  from sklearn.preprocessing import normalize
  S=normalize(S, axis=1, norm='l1')
  for i in range (S.shape[0]):
    S[i,i]=20/a[i] 
"""

" \n  a=S.sum(1)   #we will use these values, to normalize the value 20 (threshold) for each query, \n              #and we will put it in S[i,i]\n\n  from sklearn.preprocessing import normalize\n  S=normalize(S, axis=1, norm='l1')\n  for i in range (S.shape[0]):\n    S[i,i]=20/a[i] \n"

In [133]:
from numpy.lib import row_stack
#we will add a new column to df, "Similar_Queries"
#which will contain a matrix of the most similar queries to each query, and the similarity values
#These values will be normalized, and will be used as weights when calculating the missing ratings

def similar(S, df) :
  a=[]
  for i in range (S.shape[0]):
    b=[]
    for j in range (S.shape[0]):
      if S[i,j]<20 :
        b.append([j,S[i,j]])
    a.append(b)
  df['Similar_Queries']=a

  df['Similar_Queries']=df['Similar_Queries'].apply(lambda x: np.array(x).reshape(len(x),2), 1)

  for i in range (df.shape[0]):
    if (len(df['Similar_Queries'][i][:,1].reshape(-1, 1))!=0):
      df['Similar_Queries'][i][:,1]=normalize(df['Similar_Queries'][i][:,1].reshape(-1, 1), axis=0, norm='l1').reshape(1,-1)
  
  return (df)

In [205]:
#for a query that we want to rate, for a random user
#Similar queries that are not rated 

<h4>Transform the Queries definition into a table, similar to the query content table 
</h4>

In [120]:
df2= pd.read_csv("/content/data_mining_2022/data/size_30/queries.csv")

In [121]:
df2['content']=df2['content'].apply(lambda x: x.strip().split('AND'))

def f1(x):
  for i in range (len(x)):
    x[i]=x[i].strip().split('=')
  return x 
df2['content']=df2['content'].apply(lambda x: f1(x) ,0 )

df2['content']=df2['content'].apply(lambda x: np.array(x).reshape(len(x),2), 0)

def f2(x,c):
  if c in x[:,0]:
    return (x[int((np.where(x==c))[0]),1])
  else :
    return('Null')

df2['Surname']=df2['content'].apply(lambda x: f2(x,'Surname'), 0)
df2['Name']=df2['content'].apply(lambda x: f2(x,'Name'), 0)
df2['City']=df2['content'].apply(lambda x: f2(x,'City'), 0)
df2['Profession']=df2['content'].apply(lambda x: f2(x,'Profession'), 0)
df2['Age']=df2['content'].apply(lambda x: f2(x,'Age'), 0)

df2['Age']=df2['Age'].apply(lambda x: int(x) if x!='Null' else x, 0)

df2=df2.set_index ('id')

In [143]:
S=S_Matrix (df2, 5)
df2=similar(S, df2)

In [135]:
df2

,content,Surname,Name,City,Profession,Age,Similar_Queries
id,,,,,,,
query_01,"[[Profession, architect], [City, Rome], [Age, ...",Null,Ainsley,Rome,architect,38,"[[2.0, 0.06896551724137931], [13.0, 0.32758620..."
query_02,"[[Profession, pharmacist], [City, Moscow]]",Null,Null,Moscow,pharmacist,Null,[]
query_03,"[[Profession, advertising executive], [Age, 34]]",Null,Null,Null,advertising executive,34,"[[0.0, 0.08], [13.0, 0.3], [16.0, 0.1], [18.0,..."
query_04,"[[Profession, financial analyst], [Age, 90]]",Null,Null,Null,financial analyst,90,"[[5.0, 0.21428571428571427], [7.0, 0.5], [12.0..."
query_05,"[[Profession, agricultural engineer], [Surname...",Johnson,Null,Null,agricultural engineer,Null,[]
query_06,"[[Profession, teacher], [Surname, Sanders], [A...",Sanders,Annie,Null,teacher,96,"[[3.0, 0.21428571428571427], [7.0, 0.285714285..."
query_07,"[[Profession, psychologist], [City, Dublin]]",Null,Null,Dublin,psychologist,Null,[]
query_08,"[[City, Odessa], [Surname, Hughes], [Age, 104]]",Hughes,Null,Odessa,Null,104,"[[3.0, 0.6363636363636364], [5.0, 0.3636363636..."
query_09,"[[City, Tirana]]",Null,Null,Tirana,Null,Null,[]
